In [1]:
# Imports

import pandas as pd
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import the data

train = pd.read_csv('../data/train.csv', encoding='utf-8')

In [3]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [4]:
train.isnull().sum()

tweet_id     0
tweet        1
sentiment    0
dtype: int64

In [5]:
train.dropna(inplace=True)

In [6]:
train.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [7]:
# Delete ItemID

#train.drop('ItemID', axis=1, inplace=True)

In [8]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [9]:
import re
from string import punctuation

In [42]:
train['tweet'].head(10)

0           apple defining language of touch with d...
1    learning ab google doodles  all doodles should...
2    one of the most in your face ex  of stealing t...
3    this iphone    app would b pretty awesome if i...
4    line outside the apple store in   waiting for ...
5     technews one lone dude awaits ipad 2 at apple...
6      tips  prince  npr videos  toy shopping with ...
7    nu user     new  ubersocial for  iphone now in...
8               free    sampler on itunes    freemusic
9    i think i might go all weekend without seeing ...
Name: tweet, dtype: object

In [11]:
all_sentiments1 = '\n'.join(train['tweet'])

In [43]:
# all_sentiments1

In [13]:
def remove_hash(text):
    hash_pat = r'#'
    return re.sub(hash_pat, ' ', text)
def remove_mention(text):
    mention_pat = r'@mention|@[a-z]+'
    return re.sub(mention_pat, ' ', text, flags=re.I)
def remove_short_link(text):
    short_link_pat = r"bit\.ly/[a-z0-9/\-:\.=%;,\+\*())&\$!@\[\]#\?~_\.']*"
    return re.sub(short_link_pat, ' ', text, flags=re.I) 
def remove_http_link(text):
    link_permit = r"[a-z0-9/\-:\.=%;,\+\*())&\$!@\[\]#\?~_\.']"
    http_link_pat = r"http[s]?://"+link_permit+"+|//"+link_permit+"+|[\w\.]+\.[a-z]+/"+link_permit+"+" 
    return re.sub(http_link_pat, ' ', text, flags=re.I)
def remove_sub_link(text):
    link_pat = r'{link}'
    return re.sub(link_pat, ' ', text, flags=re.I)
def remove_html_char(text):
    html_char_pat = r'&[a-z]+;'
    return re.sub(html_char_pat, ' ', text, flags=re.I)    
def remove_date(text):
    pipe = r'|'
    date_pat_mon = str()
    months =   [r'January',
                r'February',
                r'March',
                r'April',
                r'May',
                r'June',
                r'July',
                r'August',
                r'September',
                r'October',
                r'November',
                r'December']
    for month in months:
        date_pat_mon = date_pat_mon + month+r' \d\d, \d\d\d\d|'
        date_pat_mon = date_pat_mon + month[:3]+r' \d\d, \d\d\d\d|'
    date_pat_mon = date_pat_mon[:-1]
#     date_pat_mon
    date_pat =  r'\d\d/\d\d/\d\d\d\d|\d\d/\d\d/\d\d' + pipe + \
                r'\d\d\.\d\d\.\d\d\d\d|\d\d\.\d\d\.\d\d' + pipe + \
                r'\d\d-\d\d-\d\d\d\d|\d\d-\d\d-\d\d' + pipe + \
                r'{}'.format(date_pat_mon)
    date_pat = r'{}'.format(date_pat)
    return re.sub(date_pat, ' ', text, flags=re.I)
def remove_short_date(text):
    pipe = r'|'
    short_date_pat = r'[\d]?\d/\d\d[\d\d]?' + pipe + r'[\d]?\d\.\d\d[\d\d]?'
    short_date_pat = r'{}'.format(short_date_pat)
    return re.sub(short_date_pat, ' ', text)
def remove_time(text):
    pipe = r'|'
    time_pat =  r'\d\d:\d\d:\d\d[ ]?pm'+pipe+\
                r'\d\d:\d\d:\d\d[ ]?am'+pipe+\
                r'\d\d:\d\d:\d\d'+pipe+\
                r'\d\d:\d\d:\d\d'+pipe+\
                r'[\d]?\d:\d\d[ ]?pm'+pipe+\
                r'[\d]?\d:\d\d[ ]?am'+pipe+\
                r'[\d]?\d:\d\d'+pipe+\
                r'[\d]?\d:\d\d'+pipe+\
                r'[\d]?\d.\d\d[ ]?pm'+pipe+\
                r'[\d]?\d.\d\d[ ]?am'+pipe+\
                r'[\d]?\d.\d\d'+pipe+\
                r'[\d]?\d.\d\d'
    time_pat= r'{}'.format(time_pat)
    return re.sub(time_pat,' ', text, flags=re.I)

def remove_sxsw(text):
    sxsw_pat = r"sxsw[a-z]*"
    return re.sub(sxsw_pat, ' ', text, flags=re.I) 

def remove_austin_texas(text):
    austin_pat = r"austin"
    texas_pat = r"texas|tx"
    temp = text
    temp = re.sub(austin_pat, ' ', temp, flags=re.I) 
    temp = re.sub(texas_pat, ' ', temp, flags=re.I) 
    return temp
def remove_punctuation(text):
    punctuation_pat_s = r'\'s'
    punctuation_pat_t = r'\'t'
    punctuation_pat_d = r'\'d'
    punctuation_pat_ve = r'\'ve'
    punctuation_pat_ll = r'\'ll'
    temp = text
    temp = re.sub(punctuation_pat_s, ' ', temp, flags=re.I)
    temp = re.sub(punctuation_pat_t, 't', temp, flags=re.I)
    temp = re.sub(punctuation_pat_d, ' would', temp, flags=re.I)
    temp = re.sub(punctuation_pat_ve, ' have', temp, flags=re.I)
    temp = re.sub(punctuation_pat_ll, ' will', temp, flags=re.I)
    return temp
def remove_rt(text):
    rt_pat = r'RT'
    return re.sub(rt_pat, ' ', text)


def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return re.sub(emoji_pattern,'', text)
def remove_not_alnum(text):
    """
    remove everything that is not alpha numeric
    """
    only_text = r'[^a-z0-9 ]'
    return re.sub(only_text, ' ', text, flags=re.I)  



In [14]:
def clean_text(text):
    #replace links witn {link}
    text = remove_short_link(text)
    text = remove_http_link(text)
    #remove {link}
    text = remove_sub_link(text)
    #hash
    text = remove_hash(text)
    #mention
    text = remove_mention(text)
    #rt
    text = remove_rt(text)
    #html spl chars
    text = remove_html_char(text)
    #time
    text = remove_time(text)
    #svxm
    text=remove_sxsw(text)
    #texas
    text=remove_austin_texas(text)
    #date
    text = remove_date(text)
    text = remove_short_date(text)
    #punctuation
    text = remove_punctuation(text)
    #emotjis
    text = remove_emojis(text)
    #remove spl chars
    text = remove_not_alnum(text)
    #return lower
    return text.lower()

In [45]:

# def performCleansingForSentimentalAnalysisWithStopWords(completeText):
# #     print('##### Get Tokens in Lower Case #######')
# #     #Get tokens in lower case
# #     lowercasetokens = getUniqueTokens(completeText.lower())
# #     unique_lowercase_tokens = len(set(lowercasetokens))
# #     print('Lowercase Unique Tokens are ' + format(unique_lowercase_tokens))
# #     print('##### Applying  stemming #######')
# #     #Applying the stemming
# #     unique_lowercase_tokens_stemmed= stemData(lowercasetokens)
# #     len_unique_lowercase_tokens_stemmed = len(unique_lowercase_tokens_stemmed)
# #     print('Lowercase Unique Tokens after steming  ' + format(len_unique_lowercase_tokens_stemmed))
# #     print('##### Applying stop words + removing Hashtags + mentions + Links + Short Links +  #######')
#  #   hashtags = find_hashtags(lowercompleteText)
#  #    mentions = find_mentions(lowercompleteText)
#  #   links = find_links(lowercompleteText)
# #    numbers = find_numbers(lowercompleteText)
# #    emojis = find_emojis(lowercompleteText)
# #    punctuations = list(punctuation)
# #     unique_lowercase_tokens_stemmed_without_stopwords = removeCustomStopWords(lowercompleteText,hashtags,mentions,links,numbers,emojis,punctuations)
# #     return unique_lowercase_tokens_stemmed_without_stopwords
#     lowercompleteText = completeText.lower()
#     import re
#     hastag_pat = r'#[0-9a-z_]+'
#     mentions_pat = r'\@[0-9a-z]+'
#     number_pat = r'-?\d+\.?\d+|-?\d+'
#     links_pat= r'http://\S+|https://\S+'
#     punctuations_pat= r'[.!?\\-]'
#     #short_links_pat =
#     emoji_pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags=re.UNICODE)
#     def remove_hashtags(tweet):
#         return re.sub(hastag_pat, '', tweet)
#     def remove_mentions(tweet):
#         return re.sub(mentions_pat, '', tweet)
#     def remove_numbers(tweet):
#         return re.sub(number_pat, '',tweet)
#     def remove_links(tweet):
#         return re.sub(links_pat, '', tweet)
#     def remove_emojis(tweet):
#         return re.sub(emoji_pattern, '', tweet)
#     def remove_shortLinks(tweet):
#         return re.sub(short_links_pat, '', tweet)
#     def remove_punctuations(tweet):
#         return re.sub(punctuations_pat, '', tweet)
#     def find_hashtags(tweet):
#         return re.findall(hastag_pat, tweet)
#     def find_mentions(tweet):
#         return re.findall(mentions_pat, tweet, flags=re.I)
#     def find_links(tweet):
#         return re.findall(links_pat, tweet, flags=re.I)
#     def find_numbers(tweet):
#         return re.findall(number_pat, tweet)
#     def find_emojis(tweet):
#         return re.findall(emoji_pattern, tweet)
    
          
#     lowercompleteText = remove_hashtags(lowercompleteText)
#     lowercompleteText = remove_mentions(lowercompleteText)
#     lowercompleteText = remove_numbers(lowercompleteText)
#     lowercompleteText = remove_links(lowercompleteText)
#     lowercompleteText = remove_emojis(lowercompleteText)
#     lowercompleteText = remove_punctuations(lowercompleteText)
#   #  lowercompleteText = getstemData(lowercompleteText)
#     return  lowercompleteText


In [16]:
train['tweet'] = train['tweet'].apply(lambda x:clean_text(x))

In [46]:
train['tweet'].head(10)

0           apple defining language of touch with d...
1    learning ab google doodles  all doodles should...
2    one of the most in your face ex  of stealing t...
3    this iphone    app would b pretty awesome if i...
4    line outside the apple store in   waiting for ...
5     technews one lone dude awaits ipad 2 at apple...
6      tips  prince  npr videos  toy shopping with ...
7    nu user     new  ubersocial for  iphone now in...
8               free    sampler on itunes    freemusic
9    i think i might go all weekend without seeing ...
Name: tweet, dtype: object

In [18]:
all_sentiments = '\n'.join(train['tweet'])

In [37]:
# all_sentiments

In [20]:
all_sentiments[:50]

'       apple defining language of touch with diffe'

In [21]:
#nltk.download('stopwords')

In [22]:
#from nltk.corpus import stopwords
import nltk

from string import punctuation
from nltk.stem.porter import *
#punc = list(punctuation)

# tokenize
train['tokenized_text'] = [nltk.word_tokenize(x) for x in train['tweet']]


In [23]:
#train['tokenized_text'] = train['tokenized_text'].apply(lambda row: [word for word in row if word not in punc])

#train['tokenized_text'] = train['tokenized_text'].apply(lambda row: [word for word in row if word not in punc])

#train['tweet'] = train['tweet'].apply(lambda row: [word for word in row if word not in punc])

In [39]:
train['tokenized_text'].head(10)

0    apple defining language of touch with differen...
1    learning ab google doodles all doodles should ...
2    one of the most in your face ex of stealing th...
3    this iphone app would b pretty awesome if it d...
4    line outside the apple store in waiting for th...
5    technews one lone dude awaits ipad 2 at apple ...
6    tips prince npr videos toy shopping with zucke...
7    nu user new ubersocial for iphone now in the a...
8                     free sampler on itunes freemusic
9    i think i might go all weekend without seeing ...
Name: tokenized_text, dtype: object

In [25]:
#stemmer = PorterStemmer()

#train['tokenized_text'] = train['tokenized_text'].apply(lambda x: [stemmer.stem(i) for i in x])
train['tokenized_text'] = train['tokenized_text'].apply(lambda x: ' '.join(x))


In [40]:
train['tokenized_text'].head(10)

0    apple defining language of touch with differen...
1    learning ab google doodles all doodles should ...
2    one of the most in your face ex of stealing th...
3    this iphone app would b pretty awesome if it d...
4    line outside the apple store in waiting for th...
5    technews one lone dude awaits ipad 2 at apple ...
6    tips prince npr videos toy shopping with zucke...
7    nu user new ubersocial for iphone now in the a...
8                     free sampler on itunes freemusic
9    i think i might go all weekend without seeing ...
Name: tokenized_text, dtype: object

In [27]:

from gensim.models import Word2Vec

tokenized_tweet =train['tokenized_text'].apply(lambda x: x.split()) 

model_w2v = Word2Vec(tokenized_tweet, size=200,  window=5)

In [41]:
tokenized_tweet.head(10)

0    [apple, defining, language, of, touch, with, d...
1    [learning, ab, google, doodles, all, doodles, ...
2    [one, of, the, most, in, your, face, ex, of, s...
3    [this, iphone, app, would, b, pretty, awesome,...
4    [line, outside, the, apple, store, in, waiting...
5    [technews, one, lone, dude, awaits, ipad, 2, a...
6    [tips, prince, npr, videos, toy, shopping, wit...
7    [nu, user, new, ubersocial, for, iphone, now, ...
8               [free, sampler, on, itunes, freemusic]
9    [i, think, i, might, go, all, weekend, without...
Name: tokenized_text, dtype: object

In [29]:
model_w2v.wv.most_similar("apple")

[('popup', 0.9994679689407349),
 ('up', 0.999047040939331),
 ('shop', 0.997204601764679),
 ('pop', 0.9965310096740723),
 ('store', 0.9961819052696228),
 ('line', 0.9960314035415649),
 ('temp', 0.9941309690475464),
 ('ipad2', 0.9924941062927246),
 ('opened', 0.9911863803863525),
 ('open', 0.9899060726165771)]

In [30]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model_w2v.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


In [31]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [32]:
# Calculating average feature vector for training set
num_features = 200
trainDataVecs = getAvgFeatureVecs(tokenized_tweet, model_w2v, num_features)

Review 0 of 7273


/Users/Aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 1000 of 7273
Review 2000 of 7273
Review 3000 of 7273
Review 4000 of 7273
Review 5000 of 7273
Review 6000 of 7273
Review 7000 of 7273


/Users/Aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [33]:
trainDataVecs

array([[-0.4564567 , -0.02529931,  0.22828756, ..., -0.3126602 ,
        -0.16492665,  0.1218187 ],
       [-0.3435992 , -0.07677873,  0.14203027, ..., -0.25336492,
        -0.14194435,  0.24446253],
       [-0.54228795, -0.03029048,  0.27312258, ..., -0.38668218,
        -0.18972911,  0.1317234 ],
       ...,
       [-0.43778637, -0.09727477,  0.17651181, ..., -0.343147  ,
        -0.16840903,  0.28150856],
       [-0.48437226, -0.07635547,  0.21842796, ..., -0.3936292 ,
        -0.16865684,  0.22171764],
       [-0.45458764, -0.0638883 ,  0.2057078 , ..., -0.3417811 ,
        -0.16296116,  0.17400773]], dtype=float32)

In [34]:
pd.DataFrame(trainDataVecs).isnull().sum().head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64

In [35]:
# .isnull().sum()

In [47]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

xg = XGBClassifier()
    
print("Fitting random forest to training data....")    
xgb = xg.fit(trainDataVecs, train["sentiment"])
    

Fitting random forest to training data....


In [ ]:
pred

In [ ]:

test = pd.read_csv('test.csv', encoding='utf-8')

In [ ]:
test.head()

In [ ]:
test['tweet'] = test['tweet'].apply(lambda x:clean_text(x))

In [ ]:
test['tokenized_text'] = [nltk.word_tokenize(x) for x in test['tweet']]

In [ ]:
test['tokenized_text'] = test['tokenized_text'].apply(lambda x: ' '.join(x))


In [ ]:
test['tokenized_text'] 

In [ ]:
tokenized_tweet_test =test['tokenized_text'].apply(lambda x: x.split()) 

In [ ]:
testDataVecs = getAvgFeatureVecs(tokenized_tweet_test,  model_w2v, num_features)

In [ ]:
result = forest.predict(testDataVecs)
output = pd.DataFrame(data={"id":test["tweet_id"], "sentiment":result})
output.to_csv( "output.csv", index=False, quoting=3 )

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X = cv.fit_transform(train['tokenized_text']).toarray()
y = train['sentiment']

In [ ]:
all_words ='\n'.join([str(text) for text in train['tokenized_text']])
all_words
#all_words = '\n'.join(train['tokenized_text'])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# all the tweets

# generate wordcloud object
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

# plot wordcloud
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
#tok = TreebankWordTokenizer()

In [ ]:
# Find unique tokens

#unique_tokens = len(set(tok.tokenize(all_sentiments)))

#print(f'Unique unprocessed tokens - {unique_tokens}')

In [ ]:
# Unique lower cased tokens

#unique_lower_case = [t.lower() for t in tok.tokenize(all_sentiments)]
#unique_lc_count = len(set(unique_lower_case))

#print(f'Unique lower case unprocessed tokens - {unique_lc_count}')

In [ ]:
#ps = PorterStemmer()

In [ ]:
# Unique lower cased stemmed tokens

#unique_lc_stemmed = [ps.stem(t) for t in unique_lower_case]
#unique_lc_stem_count = len(set(unique_lc_stemmed))

#print(f'Unique lower case stemmed tokens - {unique_lc_stem_count}')

In [ ]:

# Load stop words

#stw = stopwords.words('english')

In [ ]:
# Unique lower cased stemmed w/o stop wordstokens

#unique_lc_stem_wo_stpwrds = [t for t in unique_lc_stemmed if t not in stw]
#unique_lc_stem_wo_stw_count = len(set(unique_lc_stem_wo_stpwrds))

#print(f'Unique lower case stemmed tokens without stop words - {unique_lc_stem_wo_stw_count}')

In [ ]:
# Create a word cloud

#wc = WordCloud(stopwords=stw, background_color='white', max_words=500).generate(all_sentiments.lower())

In [ ]:
#%matplotlib inline
#plt.figure(figsize=(15, 10))
#plt.clf()
#plt.imshow(wc)
#plt.axis('off')
#plt.show()

## Sentiment Analysis

In [ ]:
# Sentiment Analysis imports

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
# Sentiment categories

plt.figure(figsize=(7, 7))
sns.countplot(train['sentiment'])
plt.title('Target Variable Distribution', fontsize=16, color='navy')
plt.show()

In [ ]:
# Create X and Y
#train['tokenized_text'] = [nltk.word_tokenize(x) for x in train['tokenized_text']]

#X = train['tokenized_text']


In [ ]:
X.head()

In [ ]:
#from imblearn.over_sampling import SMOTE


In [ ]:
#sm=SMOTE(sampling_strategy=0.25,random_state=10)

In [ ]:
y.value_counts(normalize=True)*100

In [ ]:
#X_res, y_res = sm.fit_resample(X, y)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Split data in to train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)

In [ ]:
X_train

In [ ]:
#cv = CountVectorizer()

In [ ]:
#X_train_vec = cv.fit_transform(X_train)

In [ ]:
#from imblearn.over_sampling import RandomOverSampler


In [ ]:
# Creating count vectorizer from training data



In [ ]:
# Instantiate all the models

rf = RandomForestClassifier()
lr = LogisticRegression()
#gb = GradientBoostingClassifier()
xgb= XGBClassifier()

In [ ]:
# Convert my test data into vectors for prediction

#X_test_vec = cv.transform(X_test)

In [ ]:
from sklearn import metrics
from xgboost import XGBClassifier


In [ ]:
# Train the models and check CLF report



for model, name in zip([rf, lr, xgb], ['Random Forest', 'Logistic Regression', 'xgboost']):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'CLF report for {name}')
    score = metrics.f1_score(y_test, y_pred,average='weighted')
    print(score)
    print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

classifier=MultinomialNB(alpha=0.1)

In [ ]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.f1_score(y_test, y_pred,average='weighted')
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))
    print(classification_report(y_test, y_pred))

In [ ]:
tf = TfidfVectorizer()


In [ ]:
from sklearn import metrics


In [ ]:
X_train_vec_tf = tf.fit_transform(X_train)
X_test_vec_tf = tf.transform(X_test)

In [ ]:
# Train the models and check CLF report for TFIDF vectorizer

for model, name in zip([rf, lr, gb], ['Random Forest', 'Logistic Regression', 'Gradient Boosting']):
    model.fit(X_train_vec_tf, y_train)
    y_pred = model.predict(X_test_vec_tf)
    print(f'CLF report for {name}')
    score = metrics.f1_score(y_test,y_pred,average='weighted')
    print("f1 score:   %0.3f" % score)
    print(classification_report(y_test, y_pred))

## Cleaning with RE

**List of unwanted items**
1. Hashtags (#)
2. Mentions (@)
3. Links (http://, https://)
4. Short link (t.co, bit.ly)
5. Numbers
6. Emojis
7. Emoticons
8. Punctuations